## Scan with schedule file

Testing ways to simulate scanning from the Atacama desert using a schedule file

In [ ]:
import os, pickle, time
import numpy as np
import healpy as hp
from beamconv import ScanStrategy, tools
import matplotlib.pyplot as plt
import qpoint as qp
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
def get_cls(fname='../ancillary/wmap7_r0p03_lensed_uK_ext.txt'):
    cls = np.loadtxt(fname, unpack=True) # Cl in uK^2
    return cls[0], cls[1:]

In [ ]:
# Load up alm                                                                            
lmax = 700
ell, cls = get_cls()
np.random.seed(39)
alm = hp.synalm(cls, lmax=lmax, new=True, verbose=True) # uK  

In [ ]:
#nsamp      = 1000
#mmax       = 5
#fwhm       = 30.
#scan_speed = 2.8

#beam_opts = dict(lmax=800,
#                 fwhm=fwhm,       # Gaussian co-pol beam, so only specify FWHM (arcmin)
#                 btype='Gaussian')

#soscan = ScanStrategy(duration=nsamp,
#                  sample_rate=10.21,
#                  location='atacama')

#soscan.create_focal_plane(nrow=1, ncol=1, fov=1, 
#                      **beam_opts)

#soscan.allocate_maps(nside=256)

#scan_opts = dict(                 
#                 scan_speed=scan_speed,
#                 ctime_func=soscan.schedule_ctime,
#                 q_bore_func=soscan.schedule_scan)

#ctime0 = [1510000000]
#soscan.ctime_starts = ctime0
#soscan.az0s = np.array([214.98])
#soscan.az1s = np.array([250.28])
#soscan.els = np.array([53.09])
#soscan.t0s = np.array([58484.000694])
#soscan.t1s = np.array([58484.055556])

#soscan.ctime = soscan.schedule_ctime(start=0, end=nsamp, cidx=0)

#az, el, lon, lat, qbore = soscan.schedule_scan(return_all=True)

#ra, dec, _ = sat.quat2radecpa(qbore)

In [ ]:
# init scan strategy and instrument                                                      
mmax       = 5
fwhm       = 30.
scan_speed = 2.8
mlen       = 60000. 

beam_opts = dict(lmax=800,
                 fwhm=fwhm,       # Gaussian co-pol beam, so only specify FWHM (arcmin)
                 btype='Gaussian')

soscan = ScanStrategy(duration=mlen,
    sample_rate=10.21, location='atacama', external_pointing=True)

soscan.create_focal_plane(nrow=2, ncol=2, fov=3, **beam_opts)

soscan.allocate_maps(nside=256)

scan_opts = dict(scan_speed=scan_speed,
                ctime_func=soscan.schedule_ctime,
                q_bore_func=soscan.schedule_scan,
                q_bore_kwargs=dict(),
                ctime_kwargs=dict(),
                )

chunks = soscan.partition_schedule_file(filename='atacama_scan_file.txt', 
    chunksize=0.05*soscan.nsamp) 

# for chunk in chunks:

#     soscan.ctime = soscan.schedule_ctime(**chunk)
#     az, el, lon, lat, qbore = soscan.schedule_scan(return_all=True, scan_speed=scan_speed, **chunk)
#     ra, dec, _ = soscan.quat2radecpa(qbore)

#     time = np.arange(len(ra))/3600./soscan.fsamp

#     print('Chunk:')
#     print('fsamp = {}'.format(soscan.fsamp))
#     print('nhours = {}'.format(soscan.nsamp/soscan.fsamp/3600.))
#     print('Mean ra is = {:.8f}'.format(np.mean(ra)))             
#     print('Std ra is = {:.8f}'.format(np.std(ra)))             
#     print('Mean dec is = {:.8f}'.format(np.mean(dec)))               
#     print('Std dec is = {:.8f}'.format(np.std(dec))) 

#     print('Mean el is = {:.8f}'.format(np.mean(el)))             
#     print('Std el is = {:.8f}'.format(np.std(el)))             


print(soscan.chunks)

In [ ]:
## Plotting up results
sns.set(font_scale=1.7)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
ax1.plot(time, ra, label='ra')
ax1.plot(time, dec, label='dec')
ax1.set_ylabel('Angle [deg]')
# ax2.plot(time, az, label='az')
ax2.plot(time, el, label='el')
ax2.set_ylabel('Angle [deg]')
#ax3.plot(time[1:], lon[1:], label='lon')
#ax3.set_ylabel('Longitude [deg]')
#ax3.plot(time, lat, label='lat')
#ax3.set_ylabel('Latitude [deg]')
#ax3.set_xlabel('Time [hours]')

for ax in [ax1, ax2]:
    ax.legend()
    
#for ax in [ax1, ax2]:
#    ax.xaxis.set_ticklabels([])

In [ ]:
# Generate timestreams with Gaussian beams
soscan.scan_instrument_mpi(alm, **scan_opts)

maps, cond = soscan.solve_for_map()

In [ ]:
cond[cond == np.inf] = hp.UNSEEN
cart_opts = dict(unit=r'[$\mu K_{\mathrm{CMB}}$]', lonra=[-60, 40], latra=[-70, -40])
hp.mollview(cond, min=2, max=5)
hp.mollview(maps[0], min=-250, max=250)
hp.cartview(maps[1], min=-5, max=5, **cart_opts)
hp.cartview(maps[2], min=-5, max=5, **cart_opts)

In [ ]:
print(soscan.az1s)
print(soscan.els)
print(np.min(time))
print(np.max(time))
len(time)